In [1]:
"""
CIFAR-10 Convolutional Neural Networks(CNN) Example
next_batch function is copied from edo's answer
https://stackoverflow.com/questions/40994583/how-to-implement-tensorflows-next-batch-for-own-data
Author : solaris33
Project URL : http://solarisailab.com/archives/2325
"""

import tensorflow as tf
import numpy as np

# CIFAR-10 µ¥ÀÌÅÍžŠ ŽÙ¿î·Îµå ¹Þ±â À§ÇÑ kerasÀÇ helper ÇÔŒöÀÎ load_data ÇÔŒöžŠ ÀÓÆ÷Æ®ÇÕŽÏŽÙ.
from tensorflow.keras.datasets.cifar10 import load_data

In [2]:
def next_batch(num, data, labels):
  '''
  `num` °³Œö žžÅ­ÀÇ ·£ŽýÇÑ »ùÇÃµé°ú ·¹ÀÌºíµéÀ» ž®ÅÏÇÕŽÏŽÙ.
  '''
  idx = np.arange(0 , len(data))
  np.random.shuffle(idx)
  idx = idx[:num]
  data_shuffle = [data[ i] for i in idx]
  labels_shuffle = [labels[ i] for i in idx]

  return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [3]:
# CNN žðµšÀ» Á€ÀÇÇÕŽÏŽÙ. 
def build_CNN_classifier(x):
  # ÀÔ·Â ÀÌ¹ÌÁö
  x_image = x

  # Ã¹¹øÂ° convolutional layer - ÇÏ³ªÀÇ grayscale ÀÌ¹ÌÁöžŠ 64°³ÀÇ Æ¯Â¡µé(feature)Àž·Î žÊÇÎ(maping)ÇÕŽÏŽÙ.
  W_conv1 = tf.Variable(tf.truncated_normal(shape=[5, 5, 3, 64], stddev=5e-2))
  b_conv1 = tf.Variable(tf.constant(0.1, shape=[64]))
  h_conv1 = tf.nn.relu(tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1], padding='SAME') + b_conv1)

  # Ã¹¹øÂ° Pooling layer
  h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')

  # µÎ¹øÂ° convolutional layer - 32°³ÀÇ Æ¯Â¡µé(feature)À» 64°³ÀÇ Æ¯Â¡µé(feature)·Î žÊÇÎ(maping)ÇÕŽÏŽÙ.
  W_conv2 = tf.Variable(tf.truncated_normal(shape=[5, 5, 64, 64], stddev=5e-2))
  b_conv2 = tf.Variable(tf.constant(0.1, shape=[64]))
  h_conv2 = tf.nn.relu(tf.nn.conv2d(h_pool1, W_conv2, strides=[1, 1, 1, 1], padding='SAME') + b_conv2)

  # µÎ¹øÂ° pooling layer.
  h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')

  # ŒŒ¹øÂ° convolutional layer
  W_conv3 = tf.Variable(tf.truncated_normal(shape=[3, 3, 64, 128], stddev=5e-2))
  b_conv3 = tf.Variable(tf.constant(0.1, shape=[128]))
  h_conv3 = tf.nn.relu(tf.nn.conv2d(h_pool2, W_conv3, strides=[1, 1, 1, 1], padding='SAME') + b_conv3)

  # ³×¹øÂ° convolutional layer
  W_conv4 = tf.Variable(tf.truncated_normal(shape=[3, 3, 128, 128], stddev=5e-2))
  b_conv4 = tf.Variable(tf.constant(0.1, shape=[128])) 
  h_conv4 = tf.nn.relu(tf.nn.conv2d(h_conv3, W_conv4, strides=[1, 1, 1, 1], padding='SAME') + b_conv4)

  # ŽÙŒž¹øÂ° convolutional layer
  W_conv5 = tf.Variable(tf.truncated_normal(shape=[3, 3, 128, 128], stddev=5e-2))
  b_conv5 = tf.Variable(tf.constant(0.1, shape=[128]))
  h_conv5 = tf.nn.relu(tf.nn.conv2d(h_conv4, W_conv5, strides=[1, 1, 1, 1], padding='SAME') + b_conv5)

  # Fully Connected Layer 1 - 2¹øÀÇ downsampling ÀÌÈÄ¿¡, ¿ìž®ÀÇ 32x32 ÀÌ¹ÌÁöŽÂ 8x8x128 Æ¯Â¡žÊ(feature map)ÀÌ µËŽÏŽÙ.
  # ÀÌžŠ 384°³ÀÇ Æ¯Â¡µé·Î žÊÇÎ(maping)ÇÕŽÏŽÙ.
  W_fc1 = tf.Variable(tf.truncated_normal(shape=[8 * 8 * 128, 384], stddev=5e-2))
  b_fc1 = tf.Variable(tf.constant(0.1, shape=[384]))

  h_conv5_flat = tf.reshape(h_conv5, [-1, 8*8*128])
  h_fc1 = tf.nn.relu(tf.matmul(h_conv5_flat, W_fc1) + b_fc1)

  # Dropout - žðµšÀÇ º¹ÀâµµžŠ ÄÁÆ®·ÑÇÕŽÏŽÙ. Æ¯Â¡µéÀÇ co-adaptationÀ» ¹æÁöÇÕŽÏŽÙ.
  h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob) 

  # Fully Connected Layer 2 - 384°³ÀÇ Æ¯Â¡µé(feature)À» 10°³ÀÇ Å¬·¡œº-airplane, automobile, bird...-·Î žÊÇÎ(maping)ÇÕŽÏŽÙ.
  W_fc2 = tf.Variable(tf.truncated_normal(shape=[384, 10], stddev=5e-2))
  b_fc2 = tf.Variable(tf.constant(0.1, shape=[10]))
  logits = tf.matmul(h_fc1_drop,W_fc2) + b_fc2
  y_pred = tf.nn.softmax(logits)

  return y_pred, logits

In [4]:
# ÀÎÇ² ŸÆ¿ôÇ² µ¥ÀÌÅÍ, µå·ÓŸÆ¿ô È®·üÀ» ÀÔ·Â¹Þ±âÀ§ÇÑ ÇÃ·¹ÀÌœºÈŠŽõžŠ Á€ÀÇÇÕŽÏŽÙ.
x = tf.placeholder(tf.float32, shape=[None, 32, 32, 3])
y = tf.placeholder(tf.float32, shape=[None, 10])
keep_prob = tf.placeholder(tf.float32)

# CIFAR-10 µ¥ÀÌÅÍžŠ ŽÙ¿î·ÎµåÇÏ°í µ¥ÀÌÅÍžŠ ºÒ·¯¿ÉŽÏŽÙ.
(x_train, y_train), (x_test, y_test) = load_data()
# scalar ÇüÅÂÀÇ ·¹ÀÌºí(0~9)À» One-hot Encoding ÇüÅÂ·Î º¯È¯ÇÕŽÏŽÙ.
y_train_one_hot = tf.squeeze(tf.one_hot(y_train, 10),axis=1)
y_test_one_hot = tf.squeeze(tf.one_hot(y_test, 10),axis=1)

# Convolutional Neural Networks(CNN) ±×·¡ÇÁžŠ »ýŒºÇÕŽÏŽÙ.
y_pred, logits = build_CNN_classifier(x)

# Cross EntropyžŠ ºñ¿ëÇÔŒö(loss function)Àž·Î Á€ÀÇÇÏ°í, RMSPropOptimizeržŠ ÀÌ¿ëÇØŒ­ ºñ¿ë ÇÔŒöžŠ ÃÖŒÒÈ­ÇÕŽÏŽÙ.
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits))
train_step = tf.train.RMSPropOptimizer(1e-3).minimize(loss)

# Á€È®µµžŠ °è»êÇÏŽÂ ¿¬»êÀ» Ãß°¡ÇÕŽÏŽÙ.
correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

170500096/170498071 [==============================] - 71s 0us/step
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [7]:
# ŒŒŒÇÀ» ¿­Ÿî œÇÁŠ ÇÐœÀÀ» ÁøÇàÇÕŽÏŽÙ.
with tf.Session() as sess:
  # žðµç º¯ŒöµéÀ» ÃÊ±âÈ­ÇÑŽÙ. 
  sess.run(tf.global_variables_initializer())
  
# 10000 StepžžÅ­ ÃÖÀûÈ­žŠ ŒöÇàÇÕŽÏŽÙ.
  for i in range(10000):
    batch = next_batch(128, x_train, y_train_one_hot.eval())

    # 100 Stepž¶ŽÙ training µ¥ÀÌÅÍŒÂ¿¡ ŽëÇÑ Á€È®µµ¿Í lossžŠ Ãâ·ÂÇÕŽÏŽÙ.
    if i % 100 == 0:
      train_accuracy = accuracy.eval(feed_dict={x: batch[0], y: batch[1], keep_prob: 1.0})
      loss_print = loss.eval(feed_dict={x: batch[0], y: batch[1], keep_prob: 1.0})

      print("¹Ýº¹(Epoch): %d, Æ®·¹ÀÌŽ× µ¥ÀÌÅÍ Á€È®µµ: %f, ŒÕœÇ ÇÔŒö(loss): %f" % (i, train_accuracy, loss_print))
    # 20% È®·üÀÇ DropoutÀ» ÀÌ¿ëÇØŒ­ ÇÐœÀÀ» ÁøÇàÇÕŽÏŽÙ.
    sess.run(train_step, feed_dict={x: batch[0], y: batch[1], keep_prob: 0.8})
      # ÇÐœÀÀÌ ³¡³ªžé Å×œºÆ® µ¥ÀÌÅÍ(10000°³)¿¡ ŽëÇÑ Á€È®µµžŠ Ãâ·ÂÇÕŽÏŽÙ.  
  test_accuracy = 0.0  
  for i in range(10):
    test_batch = next_batch(1000, x_test, y_test_one_hot.eval())
    test_accuracy = test_accuracy + accuracy.eval(feed_dict={x: test_batch[0], y: test_batch[1], keep_prob: 1.0})
  test_accuracy = test_accuracy / 10;
  print("Å×œºÆ® µ¥ÀÌÅÍ Á€È®µµ: %f" % test_accuracy)

¹Ýº¹(Epoch): 0, Æ®·¹ÀÌŽ× µ¥ÀÌÅÍ Á€È®µµ: 0.109375, ŒÕœÇ ÇÔŒö(loss): 106.030365
¹Ýº¹(Epoch): 100, Æ®·¹ÀÌŽ× µ¥ÀÌÅÍ Á€È®µµ: 0.156250, ŒÕœÇ ÇÔŒö(loss): 2.259440
¹Ýº¹(Epoch): 200, Æ®·¹ÀÌŽ× µ¥ÀÌÅÍ Á€È®µµ: 0.265625, ŒÕœÇ ÇÔŒö(loss): 2.019624
¹Ýº¹(Epoch): 300, Æ®·¹ÀÌŽ× µ¥ÀÌÅÍ Á€È®µµ: 0.234375, ŒÕœÇ ÇÔŒö(loss): 2.039469
¹Ýº¹(Epoch): 400, Æ®·¹ÀÌŽ× µ¥ÀÌÅÍ Á€È®µµ: 0.343750, ŒÕœÇ ÇÔŒö(loss): 1.771991
¹Ýº¹(Epoch): 500, Æ®·¹ÀÌŽ× µ¥ÀÌÅÍ Á€È®µµ: 0.484375, ŒÕœÇ ÇÔŒö(loss): 1.494523
¹Ýº¹(Epoch): 600, Æ®·¹ÀÌŽ× µ¥ÀÌÅÍ Á€È®µµ: 0.421875, ŒÕœÇ ÇÔŒö(loss): 1.586675
¹Ýº¹(Epoch): 700, Æ®·¹ÀÌŽ× µ¥ÀÌÅÍ Á€È®µµ: 0.468750, ŒÕœÇ ÇÔŒö(loss): 1.582230
¹Ýº¹(Epoch): 800, Æ®·¹ÀÌŽ× µ¥ÀÌÅÍ Á€È®µµ: 0.523438, ŒÕœÇ ÇÔŒö(loss): 1.348082
¹Ýº¹(Epoch): 900, Æ®·¹ÀÌŽ× µ¥ÀÌÅÍ Á€È®µµ: 0.515625, ŒÕœÇ ÇÔŒö(loss): 1.419751
¹Ýº¹(Epoch): 1000, Æ®·¹ÀÌŽ× µ¥ÀÌÅÍ Á€È®µµ: 0.523438, ŒÕœÇ ÇÔŒö(loss): 1.336760
¹Ýº¹(Epoch): 1100, Æ®·¹ÀÌŽ× µ¥ÀÌÅÍ Á€È®µµ: 0.546875, ŒÕœÇ ÇÔŒö(loss): 1.265695
¹Ýº¹(Epoch): 1200, Æ®·¹ÀÌŽ× µ¥ÀÌÅÍ Á€È®µµ: 0.523438, ŒÕœÇ ÇÔŒö